In [1]:
import pandas as pd

In [2]:
Data = pd.read_csv(r"C:\Users\Carlo\Desktop\Todo el proyecto\Data Posición\Porteros.csv")
pd.set_option('display.max_columns', None)

In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

Columna_habilidades = Data.columns.drop(["Name", "League"])

features = Data[Columna_habilidades]

# Crea un objeto StandardScaler
scaler = StandardScaler()

# Ajusta el escalador a tus datos y luego transforma tus datos
features_scaled = scaler.fit_transform(features)

# Inicializa el modelo NearestNeighbors
model = NearestNeighbors(n_neighbors=6, algorithm = "auto")  # Buscamos 6 vecinos porque uno de ellos será el jugador mismo

# Ajusta el modelo a tus datos
model.fit(features_scaled)

caracteristicas_grafico = ['Saves', 'Clean sheets', 'Total duels won %', 'Penalties saved', 'Goals conceded inside the box', 'Goals conceded outside the box', 'Accurate long balls %']

def buscar_jugadores_similares(nombre, caracteristicas_grafico):
    jugador = Data[Data['Name'] == nombre]
    if len(jugador) == 0:
        return 'Jugador no encontrado'
    else:
        jugador_scaled = scaler.transform(jugador[features.columns])
        distancias, indices = model.kneighbors(jugador_scaled)
        jugadores_similares = Data.iloc[indices[0][1:]]  # Excluimos el primer resultado porque será el jugador mismo

        # Escala todas las características de los jugadores similares
        jugadores_similares_scaled = scaler.transform(jugadores_similares[features.columns])

        # Crea un gráfico de radar para cada jugador similar
        fig = go.Figure()
        for i, row in enumerate(jugadores_similares_scaled):
            fig.add_trace(go.Scatterpolar(
                r=row[features.columns.isin(caracteristicas_grafico)],
                theta=caracteristicas_grafico,
                fill='toself',
                name=jugadores_similares.iloc[i]['Name']
            ))
        fig.update_layout(
            autosize=False,
            width=800,
            height=800,
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[-2, 10]  # Ajusta este rango según tus datos
                )),
            showlegend=True
        )
        fig.show()

        return jugadores_similares


In [4]:
buscar_jugadores_similares('Hernán Galíndez', caracteristicas_grafico)

,Name,Interceptions,Penalty committed,Errors lead to goal,Dribbled past,Total passes,Accurate passes %,Accurate long balls %,Saves,Clean sheets,Penalties faced,Penalties saved,Goals conceded inside the box,Goals conceded outside the box,Aerial duels won %,Total duels won %,Appearances,Age Range,Position,League,Level
184,Dean Lyness,0.00,0.0,0.03,0.06,27.39,56.65,33.02,3.13,0.39,0.10,0.00,0.74,0.29,77.78,73.33,31,3,1,Irlanda Premier Division,3
715,Caique França,0.00,0.0,0.00,0.03,23.53,64.34,37.87,3.11,0.42,0.14,0.00,0.78,0.19,75.00,66.67,36,2,1,Brasil Serie B,3
2,Rafael Romo,0.00,0.0,0.00,0.09,24.65,74.60,43.93,4.04,0.43,0.13,0.00,1.09,0.04,75.00,75.00,23,3,1,Ecuador Serie A,3
714,Gustavo,0.00,0.0,0.00,0.06,27.62,63.90,37.50,3.15,0.35,0.09,0.03,0.71,0.18,80.00,61.90,34,2,1,Brasil Serie B,3
711,Igor Bohn,0.05,0.0,0.00,0.05,28.32,60.83,27.30,4.55,0.36,0.14,0.05,0.95,0.18,71.43,66.67,22,2,1,Brasil Serie B,3
